In [1]:
import pandas as pd
from tqdm.auto import tqdm
import editdistance

In [2]:
df_train = pd.read_csv("../dataset/original/entity-resolution_advanced-topics-training_data.csv")
df_train.head(10)

,record_id,name,type,address,phone,email,modification,linked_id
0,10111147-T1,ArDIA ENTERPRISESmLTD.,entity,NaN,4.102971e+12,imquires@zoho.vo,typo,10111147
1,12172071-T0,ANDRÉ ALMEIDA BLANCO,officer,la Rua Pelotas; 209; apartamento 72; Bloco A; ...,NaN,NaN,typo,12172071
2,10104491-T1,MOONTA HOLDINGS LTD.,entity,NaN,6.501346e+10,NaN,typo,10104491
3,10068808,SPRINGFIELD INVESTMENTS GROUP LIMITED,entity,NaN,NaN,consulting@zoho.de,original,10068808
4,13011715,GRIGORY UVAROV,officer,127273; MOSCOW; OTRADNAJA STREET; HOUSE 2; NO....,NaN,NaN,original,13011715
5,10105687,ODIN GROUP INVESTMENTS S.A.,entity,NaN,4.966891e+10,info@outlook.cz,original,10105687
6,10016805-T2,CAPRI SOLUTIONS LIMITED,entity,NaN,NaN,help@CAPRI.SOLUTIONS.LxMITED.de,typo,10016805
7,12133421,Crossmatch S.A.,officer,"Akara building 24 De Castro Street, Wickham Ca...",NaN,CrossmatchSA@aol.gov,original,12133421
8,12126772,LAURENCE POUNTNEY LTD,officer,Akara Building 24 De Castro St; Wickham Cay I;...,NaN,support@LAURENCE.POUNTNEY.LTD.gov,original,12126772
9,11002523,"BREIT, MARCUS",intermediary,NaN,4.401803e+11,NaN,original,11002523


In [3]:
df_name = df_train[['record_id','name','linked_id']]
df_name.name = df_name.name.astype(str)
df_name.name = df_name.name.str.lower()
df_name.head(10)

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,record_id,name,linked_id
0,10111147-T1,ardia enterprisesmltd.,10111147
1,12172071-T0,andré almeida blanco,12172071
2,10104491-T1,moonta holdings ltd.,10104491
3,10068808,springfield investments group limited,10068808
4,13011715,grigory uvarov,13011715
5,10105687,odin group investments s.a.,10105687
6,10016805-T2,capri solutions limited,10016805
7,12133421,crossmatch s.a.,12133421
8,12126772,laurence pountney ltd,12126772
9,11002523,"breit, marcus",11002523


In [4]:
blacklist = [' ltd ', ' ltd.', ' ltd', 'enterprises', 'enterprise', ' limited', ' limited ', ' inc ', ' inc. ', ' inc.', 
            ' s.a. ', ' s.a.', 'international', 'holdings', 'investments', ' corp.', ' corp. ', 'corp. ', ' investment ', 
             ' investment', 'investment ', ' management ', 'management ', ' management', 'development', ' group ', ' group',
             'group ', ' trading', ' trading ', 'trading ', ' services', 'services ', ' services', ' holding', ' holding ', 
             'holding ', "  "]
for w in tqdm(blacklist):
    df_name.name = df_name.name.str.replace(w, ' ')
df_name.head(10)

,record_id,name,linked_id
0,10111147-T1,ardia mltd.,10111147
1,12172071-T0,andré almeida blanco,12172071
2,10104491-T1,moonta,10104491
3,10068808,springfield,10068808
4,13011715,grigory uvarov,13011715
5,10105687,odin,10105687
6,10016805-T2,capri solutions,10016805
7,12133421,crossmatch,12133421
8,12126772,laurence pountney,12126772
9,11002523,"breit, marcus",11002523


In [5]:
dict_dupl = {}

In [6]:
for (i,n) in tqdm(zip(df_name.record_id, df_name.name)):
    if n in dict_dupl.keys():
        dict_dupl[n].append(i)
    else:
        dict_dupl[n] = [i]

In [7]:
len(list(dict_dupl))

314869

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_name.iloc[:,:-1], 
                                                    df_name.linked_id, test_size=0.3, random_state=17)

In [9]:
X_train = X_train.sort_index()
X_train.head(10)

,record_id,name
0,10111147-T1,ardia mltd.
1,12172071-T0,andré almeida blanco
2,10104491-T1,moonta
4,13011715,grigory uvarov
5,10105687,odin
6,10016805-T2,capri solutions
7,12133421,crossmatch
8,12126772,laurence pountney
9,11002523,"breit, marcus"
10,10046375-NV0,afccotrade


In [10]:
y_train = y_train.sort_index()
y_train.head(10)

0     10111147
1     12172071
2     10104491
4     13011715
5     10105687
6     10016805
7     12133421
8     12126772
9     11002523
10    10046375
Name: linked_id, dtype: int64

In [11]:
X_test = X_test.sort_index()
X_test.head(10)

,record_id,name
3,10068808,springfield
14,12188364,li guobang
15,10151690,zucean
16,10117892-T4,longxi
19,10045048-M0,"sandro botticelli,"
20,12220469-M2,"ali ahmed darrat, najib ahmed darrat, ahmed aa..."
25,12214184,santiago peña durini
26,13012085,lio wen tong
29,10022235-M2,spirit ride
30,10198754-M1,inversiones siglo veintiuno


In [12]:
y_test = y_test.sort_index()
y_test.head(10)

3     10068808
14    12188364
15    10151690
16    10117892
19    10045048
20    12220469
25    12214184
26    13012085
29    10022235
30    10198754
Name: linked_id, dtype: int64

In [13]:
for(i, n) in tqdm(zip(X_test.index[:10], X_test.name[:10])):
    print(f'{i} {n}')
    for (rid, row) in tqdm(zip(X_train.record_id, X_train.name)):
        if (len(n) - editdistance.eval(n, row))/len(n) > 0.85:
            label = rid.split('-')[0]
            print(f'{rid} - {row}: label {label}')

3 springfield  


10206335-M1 - springfield   : label 10206335
10031869-NV0 - springfield : label 10031869
10031869 - springfield : label 10031869
10068808-T2 - springfield  : label 10068808
10206335-M0 - springfield   : label 10206335
10068808-T0 - springfield  : label 10068808
10206335 - springfield   : label 10206335

14 li guobang


12136136-NV1 - li guogang: label 12136136
12188364-NV1 - li guobang: label 12188364
12188364-NV0 - li guobang: label 12188364
12136136 - li guogang: label 12136136
12136136-NV0 - li guogang: label 12136136

15 zucean 


10151690-T2 - zucean : label 10151690
10151690-T0 - zuwean : label 10151690
10151690-T1 - zucean : label 10151690

16 longxi 


10117892-T3 - longxi : label 10117892
10117892 - longxi : label 10117892
10117892-T1 - longxi : label 10117892

19 sandro botticelli, 


...

12220469-NV1 - ali ahmed darrat, najib ahmed darrat, ahmed aa elderat and kamal ahmed darrat: label 12220469
12220469-M1 - ali ahmed darrat, najib ahmed darrat, ahmed aa elderat and kamal ahmed darrat: label 12220469
12220469 - ali ahmed darrat, najib ahmed darrat, ahmed aa elderat and kamal ahmed darrat: label 12220469
12220469-NV0 - ali ahmed darrat, najib ahmed darrat, ahmed aa elderat and kamal ahmed darrat: label 12220469

25 santiago peña durini


12214184-M1 - santiago peña durini: label 12214184

26 lio wen tong


13009669 - lio wen tong: label 13009669
13012085-NV0 - lio wen tong: label 13012085
13009669-NV2 - lio wen tong: label 13009669

29 spirit ride 


10022235-M0 - spirit ride : label 10022235
10022235-M1 - spirit ride : label 10022235
10022235 - spirit ride : label 10022235

30 inversiones siglo veintiuno 


10198754 - inversiones siglo veintiuno : label 10198754
10198754-M2 - inversiones siglo veintiuno : label 10198754




In [14]:
for(i, n) in tqdm(zip(X_test.index[:10], X_test.name[:10])):
    print(f'{i} {n}')
    for train_n in tqdm(list(dict_dupl)):
        #if (len(n) - editdistance.eval(n, train_n))/len(n) > 0.85:
        if editdistance.eval(n, train_n) < 2:
            print(f'{dict_dupl[train_n]} - {train_n}')

3 springfield  


['10068808', '10068808-T2', '10068808-T0'] - springfield  
['10206335-M1', '10206335-M0', '10206335'] - springfield   
['10031869-NV0', '10031869'] - springfield 

14 li guobang


['12188364', '12188364-NV1', '12188364-NV0'] - li guobang
['12136136-NV1', '12136136', '12136136-NV0'] - li guogang

15 zucean 


['10151690', '10151690-NV0', '10151690-NV1', '10151690-T2', '10151690-T1'] - zucean 
['10151690-T0'] - zuwean 

16 longxi 


['10117892-T4', '10117892-T2', '10117892-T3', '10117892', '10117892-T1'] - longxi 

19 sandro botticelli, 


...

['12220469-M2', '12220469-NV1', '12220469-NV2', '12220469-M1', '12220469-M0', '12220469', '12220469-NV0'] - ali ahmed darrat, najib ahmed darrat, ahmed aa elderat and kamal ahmed darrat

25 santiago peña durini


['12214184', '12214184-M1'] - santiago peña durini

26 lio wen tong


['13012085', '13009669-NV1', '13012085-NV1', '13009669', '13012085-NV0', '13009669-NV0', '13009669-NV2'] - lio wen tong
['12162463'] - lio weng tong

29 spirit ride 


['10022235-M2', '10022235-M0', '10022235-M1', '10022235'] - spirit ride 

30 inversiones siglo veintiuno 


['10198754-M1', '10198754', '10198754-M2', '10198754-M0'] - inversiones siglo veintiuno 




In [15]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [16]:
ls ../cython/editdistance/ !

ls: !: No such file or directory
ls: ../cython/editdistance/: No such file or directory


In [ ]:
from ..cython

In [17]:
for(i, n) in tqdm(zip(X_test.index[:10], X_test.name[:10])):
    print(f'{i} {n}')
    for train_n in tqdm(list(dict_dupl)):
        #if (len(n) - editdistance.eval(n, train_n))/len(n) > 0.85:
        if ed(n, train_n) < 2:
            print(f'{dict_dupl[train_n]} - {train_n}')

3 springfield  


NameError: name 'ed' is not defined